In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.diagnostic import linear_rainbow, het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.preprocessing import LabelEncoder

# Inferential Modeling Workflow

This dataset was downloaded from [Kaggle](https://www.kaggle.com/kumarajarshi/life-expectancy-who) and reflects data collected by the WHO about life expectancy and potentially-related factors.  The information is aggregated on a per-country per-year basis.

The following questions have been posed:

1. Does various predicting factors which has been chosen initially really affect the Life expectancy? What are the predicting variables actually affecting the life expectancy?
2. Should a country having a lower life expectancy value(<65) increase its healthcare expenditure in order to improve its average lifespan?
3. How does Infant and Adult mortality rates affect life expectancy?
4. Does Life Expectancy has positive or negative correlation with eating habits, lifestyle, exercise, smoking, drinking alcohol etc.
5. What is the impact of schooling on the lifespan of humans?
6. Does Life Expectancy have positive or negative relationship with drinking alcohol?
7. Do densely populated countries tend to have lower life expectancy?
8. What is the impact of Immunization coverage on life Expectancy?

### Importing the Data

In [ ]:
df = pd.read_csv("data/life_expectancy.csv")

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.columns

### Initial Data Preparation

The original column names have extra spaces and other irregularities.  Let's clean those up, and also move the target variable to be the first column, for readability

In [ ]:
# rename so everything is snake_case
df = df.rename(columns={
    'Life expectancy ': 'Life_Expectancy',
    'Adult Mortality': 'Adult_Mortality',
    'infant deaths': 'Infant_Deaths',
    'percentage expenditure': 'Percentage_Expenditure',
    'Hepatitis B': 'Hepatitis_B',
    'Measles ': 'Measles',
    ' BMI ': 'BMI',
    'under-five deaths ': 'Under_five_Deaths',
    'Total expenditure': 'Total_Expenditure',
    'Diphtheria ': 'Diptheria',
    ' HIV/AIDS': 'HIV_AIDS',
    ' thinness  1-19 years': 'Thinness_1_19_years',
    ' thinness 5-9 years': 'Thinness_5_9_years',
    'Income composition of resources': 'Income_Composition_of_Resources'
})

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
# reorder so life expectancy is the first column
cols = list(df.columns)
cols = [cols[3]] + cols[:3] + cols[4:]
df = df[cols]

In [ ]:
df.head()

### Data Understanding

There are a lot of variables here!  Let's look at a correlation matrix to see which ones might be the most useful.  (Here we are looking for variables that are highly correlated with the target variable, but not highly correlated with other input variables)

In [ ]:
corr = df.corr()
mask = np.triu(np.ones_like(corr, dtype=np.bool))

fig1, ax1 = plt.subplots(figsize=(11, 9))
sns.heatmap(corr, mask=mask, ax=ax1)

Ok, it looks like there are only a few that are highly positively correlated with life expectancy.  Let's make a pair plot of those.

(Note: we don't want to do a pair plot right from the outset because it would be too slow)

In [ ]:
positively_correlated_cols = ['Life_Expectancy','BMI', 'Income_Composition_of_Resources', 'Schooling']
positively_correlated_df = df[positively_correlated_cols]
sns.pairplot(positively_correlated_df)

### First Simple Model

Ok, it looks like the correlation with BMI is a little fuzzier than the others, so let's exclude it for now.  `Schooling` and `Income_Composition_of_Resources` are highly correlated with both life expectancy and each other, so let's only include one of them.  `Schooling` seems like a good choice because it would allow us to answer Question 5.

In [ ]:
fsm_df = df[["Schooling", "Life_Expectancy"]].copy()
fsm_df.dropna(inplace=True)

In [ ]:
fsm = ols(formula="Life_Expectancy ~ Schooling", data=fsm_df).fit()

In [ ]:
fsm.summary()

### Model Evaluation

Not too bad.  We are only explaining about 57% of the variance in life expectancy, but we only have one feature so far and it's statistically significant at an alpha of 0.05.

We could stop right now and say that according to our model:

 - A country with zero years of schooling on average is expected to have a life expectancy of 44.1 years
 - For each additional average year of schooling, we expect life expectancy to increase by 2.1 years

But before we move forward, let's also check for the assumptions of linear regression

#### Linearity

Linear regression assumes that the input variable linearly predicts the output variable.  We already qualitatively checked that with a scatter plot.  But I also think it's a good idea to use a statistical test.  This one is the [Rainbow test](https://www.tandfonline.com/doi/abs/10.1080/03610928208828423) which is available from the [diagnostic submodule of StatsModels](https://www.statsmodels.org/stable/generated/statsmodels.stats.diagnostic.linear_rainbow.html#statsmodels.stats.diagnostic.linear_rainbow)

In [ ]:
rainbow_statistic, rainbow_p_value = linear_rainbow(fsm)
print("Rainbow statistic:", rainbow_statistic)
print("Rainbow p-value:", rainbow_p_value)

The null hypothesis is that the model is linearly predicted by the features, alternative hypothesis is that it is not.  Thus returning a low p-value means that the current model violates the linearity assumption.

#### Normality

Linear regression assumes that the residuals are normally distributed.  It is possible to check this qualitatively with a Q-Q plot, but this example shows how to assess it statistically.

The [Jarque-Bera](https://en.wikipedia.org/wiki/Jarque%E2%80%93Bera_test) test is performed automatically as part of the model summary output, labeled **Jarque-Bera (JB)** and **Prob(JB)**.

The null hypothesis is that the residuals are normally distributed, alternative hypothesis is that they are not.  Thus returning a low p-value means that the current model violates the normality assumption.

#### Homoscadasticity

Linear regression assumes that the variance of the dependent variable is homogeneous across different value of the independent variable(s).  We can visualize this by looking at the predicted life expectancy vs. the residuals.

In [ ]:
y = fsm_df["Life_Expectancy"]
y_hat = fsm.predict()

In [ ]:
fig2, ax2 = plt.subplots()
ax2.set(xlabel="Predicted Life Expectancy",
        ylabel="Residuals (Actual - Predicted Life Expectancy)")
ax2.scatter(x=y_hat, y=y-y_hat, color="blue", alpha=0.2)

Just visually inspecting this, it seems like our model over-predicts life expectancy between 60 and 70 years old in a way that doesn't happen for other age groups.  Plus we have some weird-looking data in the lower end that we might want to inspect.  Maybe there was something wrong with recording those values, or maybe there is something we can feature engineer once we have more independent variables.

Let's also run a statistical test.  The [Breusch-Pagan test](https://en.wikipedia.org/wiki/Breusch%E2%80%93Pagan_test) is available from the [diagnostic submodule of StatsModels](https://www.statsmodels.org/stable/generated/statsmodels.stats.diagnostic.het_breuschpagan.html#statsmodels.stats.diagnostic.het_breuschpagan)

In [ ]:
lm, lm_p_value, fvalue, f_p_value = het_breuschpagan(y-y_hat, fsm_df[["Schooling"]])
print("Lagrange Multiplier p-value:", lm_p_value)
print("F-statistic p-value:", f_p_value)

The null hypothesis is homoscedasticity, alternative hypothesis is heteroscedasticity.  Thus returning a low p-value means that the current model violates the homoscedasticity assumption

#### Independence

The independence assumption means that the independent variables must not be too collinear.  Right now we have only one independent variable, so we don't need to check this yet.

## Adding Features to the Model

So far, all we have is a simple linear regression.  Let's start adding features to make it a multiple regression.

First, I'll repeat the process of the highly positively correlated variables, but this time with the highly negatively correlated variables (based on looking at the correlation matrix)

In [ ]:
negatively_correlated_cols = [
    'Life_Expectancy',
    'Adult_Mortality',
    'HIV_AIDS',
    'Thinness_1_19_years',
    'Thinness_5_9_years'
]
negatively_correlated_df = df[negatively_correlated_cols]
sns.pairplot(negatively_correlated_df)

`Adult_Mortality` seems most like a linear relationship to me.  Also, the two thinness metrics seem to be providing very similar information, so we almost certainly should not include both

A quick experiment to try to flatten out that curve:

In [ ]:
fig3, ax3 = plt.subplots()

ax3.set(xlabel="Adult_Mortality", ylabel="Life_Expectancy")
ax3.scatter(x=np.sqrt(df["Adult_Mortality"]), y=df["Life_Expectancy"])

This gives me straighter lines, but seems to indicate that we probably need at least two separate models to represent this data correctly.  However in the interest of time, I'm just going to assume the relationship is linear.

In [ ]:
model_2_df = df[["Life_Expectancy", "Schooling", "Adult_Mortality"]].copy()
model_2_df.dropna(inplace=True)

In [ ]:
model_2 = ols(formula="Life_Expectancy ~ Schooling + Adult_Mortality", data=model_2_df).fit()

In [ ]:
model_2.summary()

### Second Model Evaluation

Adding another feature improved the r-squared from 0.565 to 0.714

But let's also look at the model assumptions

#### Linearity

In [ ]:
rainbow_statistic, rainbow_p_value = linear_rainbow(model_2)
print("Rainbow statistic:", rainbow_statistic)
print("Rainbow p-value:", rainbow_p_value)

Assuming an alpha of 0.05, we are no longer violating the linearity assumption (just barely)

#### Normality

The **Jarque-Bera (JB)** output has gotten worse.  We are still violating the normality assumption.

#### Homoscadasticity

In [ ]:
y = model_2_df["Life_Expectancy"]
y_hat = model_2.predict()

In [ ]:
fig4, ax4 = plt.subplots()
ax4.set(xlabel="Predicted Life Expectancy",
        ylabel="Residuals (Actual - Predicted Life Expectancy)")
ax4.scatter(x=y_hat, y=y-y_hat, color="blue", alpha=0.2)

In [ ]:
lm, lm_p_value, fvalue, f_p_value = het_breuschpagan(y-y_hat, model_2_df[["Schooling", "Adult_Mortality"]])
print("Lagrange Multiplier p-value:", lm_p_value)
print("F-statistic p-value:", f_p_value)

Both visually and numerically, we can see some improvement.  But we are still violating this assumption to a statistically significant degree.

#### Independence

You might have noticed in the regression output that there was a warning about the condition number being high.  The condition number is a measure of stability of the matrix used for computing the regression (we'll discuss this more in the next module), and a number above 30 can indicate strong multicollinearity.  Our output is way higher than that.

A different (more generous) measure of multicollinearity is the [variance inflation factor](https://en.wikipedia.org/wiki/Variance_inflation_factor).  It is available from the [outlier influence submodule of StatsModels](https://www.statsmodels.org/stable/generated/statsmodels.stats.outliers_influence.variance_inflation_factor.html#statsmodels.stats.outliers_influence.variance_inflation_factor).

In [ ]:
rows = model_2_df[["Schooling", "Adult_Mortality"]].values

vif_df = pd.DataFrame()
vif_df["VIF"] = [variance_inflation_factor(rows, i) for i in range(2)]
vif_df["feature"] = ["Schooling", "Adult_Mortality"]

vif_df

A "rule of thumb" for VIF is that 5 is too high, so I think it's reasonable to say that we are not violating the independence assumption, despite the high condition number.

## Adding a Categorical Value

This is less realistic than the previous steps but I wanted to give an example

In this dataset, we have a lot of numeric values (everything in that correlation matrix), but there are a few that aren't.  One example is `Status`

In [ ]:
model_3_df = df[["Life_Expectancy", "Schooling", "Adult_Mortality", "Status"]].copy()
model_3_df.dropna(inplace=True)

In [ ]:
model_3_df["Status"].value_counts()

In [ ]:
sns.catplot(x="Status", y="Life_Expectancy", data=model_3_df, kind="box")

It looks like there is a difference between the two groups that might be useful to include

There are only two categories, so we only need a `LabelEncoder` that will convert the labels into 1s and 0s.  If there were more than two categories, we would use a `OneHotEncoder`, which would create multiple columns out of a single column.

In [ ]:
label_encoder = LabelEncoder()
status_labels = label_encoder.fit_transform(model_3_df["Status"])
status_labels

In [ ]:
label_encoder.classes_

This is telling us that "Developed" is encoded as 0 and "Developing" is encoded as 1.  This means that "Developed" is assumed at the intercept.

In [ ]:
model_3_df["Status_Encoded"] = status_labels
model_3_df.drop("Status", axis=1)

In [ ]:
model_3 = ols(formula="Life_Expectancy ~ Schooling + Adult_Mortality + Status_Encoded", data=model_3_df).fit()

In [ ]:
model_3.summary()

### Third Model Evaluation

Adding another feature improved the r-squared a tiny bit from 0.714 to 0.718

Let's look at the model assumptions again

#### Linearity

In [ ]:
rainbow_statistic, rainbow_p_value = linear_rainbow(model_3)
print("Rainbow statistic:", rainbow_statistic)
print("Rainbow p-value:", rainbow_p_value)

Another small improvement

#### Normality

The **Jarque-Bera (JB)** output has gotten slightly better.  But we are still violating the normality assumption.

#### Homoscadasticity

In [ ]:
y = model_3_df["Life_Expectancy"]
y_hat = model_3.predict()

In [ ]:
fig5, ax5 = plt.subplots()
ax5.set(xlabel="Predicted Life Expectancy",
        ylabel="Residuals (Actual - Predicted Life Expectancy)")
ax5.scatter(x=y_hat, y=y-y_hat, color="blue", alpha=0.2)

In [ ]:
lm, lm_p_value, fvalue, f_p_value = het_breuschpagan(y-y_hat, model_3_df[["Schooling", "Adult_Mortality", "Status_Encoded"]])
print("Lagrange Multiplier p-value:", lm_p_value)
print("F-statistic p-value:", f_p_value)

This metric got worse, although the plot looks fairly similar

#### Independence

In [ ]:
rows = model_3_df[["Schooling", "Adult_Mortality", "Status_Encoded"]].values

vif_df = pd.DataFrame()
vif_df["VIF"] = [variance_inflation_factor(rows, i) for i in range(3)]
vif_df["feature"] = ["Schooling", "Adult_Mortality", "Status_Encoded"]

vif_df

The VIF metrics are getting higher, which means that there is stronger multicollinearity.  But we have still not exceeded the threshold of 5.

## Summary

We started with a baseline model where the only input feature was `Schooling`.  Our baseline model had an r-squared of 0.565.  This model violated the linearity (p < 0.001), normality (p < 0.001), and homoscadasticity (p < 0.001) assumptions of linear regression.  The independence assumption was met by default because there was only one input feature.

The final model for this lesson had three input features: `Schooling`, `Adult_Mortality`, and `Status_Encoded`.  It had an r-squared of 0.718.  This model did not violate the linearity assumption (p = 0.084), but it did violate the normality (p < 0.001) and homoscedasticity (p < 0.001) assumptions.  Based on the variance inflaction factor metric, it did not violate the independence assumption.

We are able to address the following questions from above:

*1. Does various predicting factors which has been chosen initially really affect the Life expectancy? What are the predicting variables actually affecting the life expectancy?*

With only 3 features we are able to explain about 71% of the variance in life expectancy.  This indicates that these factors truly are explanatory.  More analysis is required to understand how much additional explanatory power would be provided by incorporating additional features into the model.

*3. How does Infant and Adult mortality rates affect life expectancy?*

So far we have only investigated adult mortality.  The adult mortality rate ("probability of dying between 15 and 60 years per 1000 population") has a negative correlation with life expectancy.  For each increase of 1 in the adult mortality rate, life expectancy decreases by about .03 years.

*5. What is the impact of schooling on the lifespan of humans?*

In our latest model, we find that each additional year of average schooling is associated with 1.4 years of added life expectancy.  However it is challenging to interpret whether it is schooling that is actually having the impact.  Schooling is highly correlated with `Income_Composition_of_Resources` ("Human Development Index in terms of income composition of resources") so it is very possible that schooling is the result of some underlying factor that also impacts life expectancy, rather than schooling impacting life expectancy directly.

### Appendix

Things I have not done in this lesson, but that you should consider in your project:

 - More robust cleaning (possible imputation of missing values, principled exclusion of some data)
 - Feature scaling
 - Nearest-neighbors approach (requires more complex feature engineering)
 - Pulling information from external resources
 - Removing independent variables if you determine that they are causing too high of multicollinearity
 - Setting up functions so the code is not so repetitive
 
Also, I've included a dataset called `cars.csv` if you are interested in additional practice that does not use the King County Housing Data